In [1]:
from pytube import YouTube, Playlist
from rich.pretty import pprint
import re
#import tempfile
import ffmpeg

In [2]:
video_link = "https://youtu.be/9bex5sXUFB0?si=hRjr32mmM5d01byO"
output_path = "./Downloads/"

yt = YouTube(video_link)
yt.check_availability()
video = yt.streams

HTTPError: HTTP Error 403: Forbidden

In [4]:
##Filter High Quality Video
## Get audio and video on separate tracks
highquality_audiovideo_list = video.filter(adaptive=True)
## Get Only High Quality Audio or Video
highquality_video_list = (
    video.filter(only_video=True, adaptive=True).order_by("resolution").desc()
)
highquality_audio_list = (
    video.filter(only_audio=True, adaptive=True).order_by("abr").desc()
)


# Extract Number from String. I.E. 1080 from 1080p, 160 from 160kbps
abr_pattern = r"\d+"
abr_pattern_compiled = re.compile(abr_pattern)

## Create list of dictionary contain type of audio ane video format available for audio
highquality_video_list = [
    {
        "type": highquality_audiovideo.type,
        "itag": highquality_audiovideo.itag,
        "resolution": highquality_audiovideo.resolution,
        "fps": highquality_audiovideo.fps,
        "video_codec": highquality_audiovideo.video_codec,
        "mime_type": highquality_audiovideo.mime_type,
    }
    for highquality_audiovideo in highquality_audiovideo_list
    if highquality_audiovideo.type == "video"
]
highquality_audio_list = [
    {
        "type": highquality_audiovideo.type,
        "itag": highquality_audiovideo.itag,
        "average-bitrate (ABR)": highquality_audiovideo.abr,
        "_abr": int(abr_pattern_compiled.search(highquality_audiovideo.abr).group(0)),
        "acodec": highquality_audiovideo.audio_codec,
        "mime_type": highquality_audiovideo.mime_type,
    }
    for highquality_audiovideo in highquality_audiovideo_list
    if highquality_audiovideo.type == "audio"
]
highquality_audio_sorted_list = sorted(
    highquality_audio_list, key=lambda x: x["_abr"], reverse=True
)

In [5]:
# Print High quality video list
pprint(highquality_video_list)

[
│   {
│   │   'type': 'video',
│   │   'itag': 137,
│   │   'resolution': '1080p',
│   │   'fps': 25,
│   │   'video_codec': 'avc1.640028',
│   │   'mime_type': 'video/mp4'
│   },
│   {
│   │   'type': 'video',
│   │   'itag': 248,
│   │   'resolution': '1080p',
│   │   'fps': 25,
│   │   'video_codec': 'vp9',
│   │   'mime_type': 'video/webm'
│   },
│   {
│   │   'type': 'video',
│   │   'itag': 136,
│   │   'resolution': '720p',
│   │   'fps': 25,
│   │   'video_codec': 'avc1.4d401f',
│   │   'mime_type': 'video/mp4'
│   },
│   {
│   │   'type': 'video',
│   │   'itag': 247,
│   │   'resolution': '720p',
│   │   'fps': 25,
│   │   'video_codec': 'vp9',
│   │   'mime_type': 'video/webm'
│   },
│   {
│   │   'type': 'video',
│   │   'itag': 135,
│   │   'resolution': '480p',
│   │   'fps': 25,
│   │   'video_codec': 'avc1.4d401e',
│   │   'mime_type': 'video/mp4'
│   },
│   {
│   │   'type': 'video',
│   │   'itag': 244,
│   │   'resolution': '480p',
│   │   'fps': 25,
│   │   'video_codec': 'vp9',
│   │   'mime_type': 'video/webm'
│   },
│   {
│   │   'type': 'video',
│   │   'itag': 134,
│   │   'resolution': '360p',
│   │   'fps': 25,
│   │   'video_codec': 'avc1.4d401e',
│   │   'mime_type': 'video/mp4'
│   },
│   {
│   │   'type': 'video',
│   │   'itag': 243,
│   │   'resolution': '360p',
│   │   'fps': 25,
│   │   'video_codec': 'vp9',
│   │   'mime_type': 'video/webm'
│   },
│   {
│   │   'type': 'video',
│   │   'itag': 133,
│   │   'resolution': '240p',
│   │   'fps': 25,
│   │   'video_codec': 'avc1.4d4015',
│   │   'mime_type': 'video/mp4'
│   },
│   {
│   │   'type': 'video',
│   │   'itag': 242,
│   │   'resolution': '240p',
│   │   'fps': 25,
│   │   'video_codec': 'vp9',
│   │   'mime_type': 'video/webm'
│   },
│   {
│   │   'type': 'video',
│   │   'itag': 160,
│   │   'resolution': '144p',
│   │   'fps': 25,
│   │   'video_codec': 'avc1.4d400c',
│   │   'mime_type': 'video/mp4'
│   },
│   {
│   │   'type': 'video',
│   │   'itag': 278,
│   │   'resolution': '144p',
│   │   'fps': 25,
│   │   'video_codec': 'vp9',
│   │   'mime_type': 'video/webm'
│   }
]

In [6]:
# Print High Quality audio list
pprint(highquality_audio_sorted_list)

[
│   {
│   │   'type': 'audio',
│   │   'itag': 251,
│   │   'average-bitrate (ABR)': '160kbps',
│   │   '_abr': 160,
│   │   'acodec': 'opus',
│   │   'mime_type': 'audio/webm'
│   },
│   {
│   │   'type': 'audio',
│   │   'itag': 140,
│   │   'average-bitrate (ABR)': '128kbps',
│   │   '_abr': 128,
│   │   'acodec': 'mp4a.40.2',
│   │   'mime_type': 'audio/mp4'
│   },
│   {
│   │   'type': 'audio',
│   │   'itag': 250,
│   │   'average-bitrate (ABR)': '70kbps',
│   │   '_abr': 70,
│   │   'acodec': 'opus',
│   │   'mime_type': 'audio/webm'
│   },
│   {
│   │   'type': 'audio',
│   │   'itag': 249,
│   │   'average-bitrate (ABR)': '50kbps',
│   │   '_abr': 50,
│   │   'acodec': 'opus',
│   │   'mime_type': 'audio/webm'
│   },
│   {
│   │   'type': 'audio',
│   │   'itag': 139,
│   │   'average-bitrate (ABR)': '48kbps',
│   │   '_abr': 48,
│   │   'acodec': 'mp4a.40.5',
│   │   'mime_type': 'audio/mp4'
│   }
]

In [7]:
# Download a specific folder
## P
print(yt.title)
# highquality_audiovideo_list.get_by_itag(137).download(filename_prefix="Video - ", output_path=output_path)
highquality_audiovideo_list.get_by_itag(251).download(filename_prefix="Audio - ", output_path=output_path)

Navratri Special Nonstop Garba 2024 | Gujarati Garba Songs | @DadaBhagwanMusic


'/Users/dadaniru/Documents/GitHub/Youtube-Video-Downloader/./Downloads/Audio - Navratri Special Nonstop Garba 2024  Gujarati Garba Songs  @DadaBhagwanMusic.webm'

In [8]:
name = "Navratri Special Nonstop Garba 2024"
# Combine Audio Video
ip_video = ffmpeg.input(
    # output_path + "Video - " + yt.title + '.mp4'
    output_path + f"Video - {name}" + '.mp4'
)
ip_audio = ffmpeg.input(
    #output_path + "Audio - " + yt.title + '.webm'
    output_path + f"Audio - {name}" + '.webm'
)

try:
  ffmpeg.concat(ip_video, ip_audio, v=1, a=1).output(
      output_path + name + '.mp4'
  ).run(capture_stdout=True, capture_stderr=True)
except ffmpeg.Error as e:
  print('stdout:', e.stdout.decode('utf8'))
  print('stderr:', e.stderr.decode('utf8'))
  raise e

stdout: 
stderr: ffmpeg version 7.0 Copyright (c) 2000-2024 the FFmpeg developers
  built with Apple clang version 15.0.0 (clang-1500.3.9.4)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/7.0_1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --ena

Error: ffmpeg error (see stderr output for detail)

# Convert Webm to MP3

In [7]:
import subprocess

def convert_webm_to_mp3(input_file, output_file):
    # Command to convert WEBM to MP3
    command = [
        'ffmpeg',
        '-i', input_file,     # Input file
        '-vn',                # Disable video recording
        '-acodec', 'libmp3lame',  # Set audio codec to MP3
        '-ab', '320k',        # Set audio bitrate
        output_file           # Output file
    ]

    try:
        # Execute the command
        subprocess.run(command, check=True)
        print(f"Conversion successful: {output_file}")
    except subprocess.CalledProcessError as e:
        print(f"Error during conversion: {e}")

# Example usage
input_file = output_path + 'Navratri Special Nonstop Garba 2024.webm'
input_file = '/Users/dadaniru/Downloads/Locoloader video (1).webm'
# output_file = output_path + 'Aaje Pokaro Dilthi Dadane.mp3'
output_file = '/Users/dadaniru/Downloads/Guru Tara Sharan Ma.mp3'
convert_webm_to_mp3(input_file, output_file)


ffmpeg version 7.0 Copyright (c) 2000-2024 the FFmpeg developers
  built with Apple clang version 15.0.0 (clang-1500.3.9.4)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/7.0_1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libopenvino -

Conversion successful: /Users/dadaniru/Downloads/Guru Tara Sharan Ma.mp3


[out#0/mp3 @ 0x158f07ea0] video:0KiB audio:15869KiB subtitle:0KiB other streams:0KiB global headers:0KiB muxing overhead: 0.006178%
size=   15870KiB time=00:06:46.21 bitrate= 320.0kbits/s speed=96.9x    
